# Joining Data Sets

Let us understand how to join multiple Data Sets using Spark based APIs.

* Prepare Datasets for Joins
* Starting Spark Context
* Joining Data Frames
* Overview of Joins
* Problems and Solutions

## Prepare Datasets for Joins
Let us prepare Dataset to join and get the details related to airports (origin and destination).

* Make sure airport-codes is in HDFS.

In [ ]:
%%sh
hdfs dfs -ls /public/airlines_all/airport-codes

## Starting Spark Context

Let us start spark context for this Notebook so that we can execute the code provided.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    appName('Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', '2')

* Analyze the Dataset to confirm if there is header and also how the data is structured.

In [ ]:
spark.read. \
    text("/public/airlines_all/airport-codes"). \
    show(truncate=False)

 * Data is tab separated.
 * There is header for the data set.
 * Dataset have 4 fields - **Country, State, City, IATA**
    
    
Create DataFrame airport_codes applying appropriate Schema.


In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       )

* Preview and Understand the data.

In [ ]:
airport_codes.show()

* Get schema of **airport_codes**.

In [ ]:
airport_codes.printSchema()

* Preview the data
 * Get the count of records

In [ ]:
airport_codes.count()

   * Get the count of unique records and see if it is the same as total count.

In [ ]:
airport_codes. \
    select("IATA"). \
    distinct(). \
    count()

 * If they are not equal, analyze the data and identify IATA codes which are repeated more than once.

In [ ]:
from pyspark.sql.functions import lit, count

In [ ]:
duplicate_iata_count = airport_codes. \
    groupBy("IATA"). \
    agg(count(lit(1)).alias("iata_count")). \
    filter("iata_count > 1")

In [ ]:
duplicate_iata_count.show()

 * Filter out the duplicates using the most appropriate one and discard others.

In [ ]:
airport_codes. \
    filter("IATA = 'Big'"). \
    show()

In [ ]:
airport_codes. \
    filter("!(State = 'Hawaii' AND IATA = 'Big')"). \
    show()

 * Get number of airports (IATA Codes) for each state in the US. Sort the data in descending order by count.

In [ ]:
from pyspark.sql.functions import col, lit, count

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country='USA'")

In [ ]:
airport_count_per_state = airport_codes. \
    groupBy("Country", "State"). \
    agg(count(lit(1)).alias("IATACount")). \
    orderBy(col("IATACount").desc())

In [ ]:
airport_count_per_state.show()

## Joining Data Frames

Let us understand how to join Data Frames by using some problem statements. Use 2008 January data.
* Get number of flights departed from each of the US airport.
* Get number of flights departed from each of the state.
* Get the list of airports in the US from which flights are not departed.
* Check if there are any origins in airlines data which do not have record in airport-codes.
* Get the total number of flights from the airports that do not contain entries in airport-codes.
* Get the total number of flights per airport that do not contain entries in airport-codes.

## Overview of Joins

Let us get an overview of joining Data Frames.
* Our data cannot be stored in one table. It will be stored in multiple tables and the tables might be related.
 * When it comes to transactional systems, we typically define tables based on Normalization Principles.
 * When it comes to data warehousing applications, we typically define tables using Dimensional Modeling.
 * Either of the approach data is scattered into multiple tables and relationships are defined.
 * Typically tables are related with one to one, one to many, many to many relationships.
* When we have 2 Data Sets that are related based on a common key we typically perform join.
* There are different types of joins.
 * INNER JOIN
 * OUTER JOIN (LEFT or RIGHT)
 * FULL OUTER JOIN (a LEFT OUTER JOIN b UNION a RIGHT OUTER JOIN b)
 

## Solutions - Problem 1

Get number of flights departed from each of the US airport.

In [ ]:
from pyspark.sql.functions import col, lit, count

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airlines.show()

In [ ]:
airlines.select("Year", "Month", "DayOfMonth", "Origin", "Dest", "CRSDepTime").show()

In [ ]:
airlines.count()

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country='USA'")

In [ ]:
airport_codes.count()

In [ ]:
airlines. \
    join(airport_codes, col("IATA") == col("Origin")). \
    select("Year", "Month", "DayOfMonth", airport_codes["*"], "CRSDepTime"). \
    show()

In [ ]:
airlines. \
    join(airport_codes, airport_codes.IATA == airlines["Origin"]). \
    select("Year", "Month", "DayOfMonth", airport_codes["*"], "CRSDepTime"). \
    show()

In [ ]:
airlines.join?

In [ ]:
flight_count_per_airport = airlines. \
    join(airport_codes, airport_codes.IATA == airlines.Origin). \
    groupBy("Origin"). \
    agg(count(lit(1)).alias("FlightCount")). \
    orderBy(col("FlightCount").desc())

In [ ]:
flight_count_per_airport.show()

## Solutions - Problem 2

Get number of flights departed from each of the state.

In [ ]:
from pyspark.sql.functions import col, lit, count

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country='USA'")

In [ ]:
flight_count_per_state = airlines. \
    join(airport_codes, airport_codes.IATA == airlines.Origin). \
    groupBy("State"). \
    agg(count(lit(1)).alias("FlightCount")). \
    orderBy(col("FlightCount").desc())

In [ ]:
flight_count_per_state.show()

## Solutions - Problem 3

Get the list of airports in the US from which flights are not departed.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country='USA'")

In [ ]:
airport_codes.printSchema()

In [ ]:
airports_not_used = airport_codes. \
    join(airlines, airport_codes.IATA == airlines.Origin, "left"). \
    select(airport_codes["*"], "Year", "Month", 
           "DayOfMonth", "Origin", "CRSDepTime"). \
    show()

In [ ]:
airports_not_used = airport_codes. \
    join(airlines, airport_codes.IATA == airlines.Origin, "left"). \
    filter(airlines.Origin.isNull()). \
    select('City', 'State', 'Country', 'IATA')

In [ ]:
airports_not_used = airlines. \
    join(airport_codes, airport_codes.IATA == airlines.Origin, "right"). \
    filter("Origin IS NULL"). \
    select('City', 'State', 'Country', 'IATA')

In [ ]:
airports_not_used.count()

In [ ]:
airport_codes.show()

## Solutions - Problem 4

Check if there are any origins in airlines data which do not have record in airport-codes.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big')")

In [ ]:
airlines. \
    join(airport_codes, airlines.Origin == airport_codes.IATA, "left"). \
    filter("IATA IS NULL"). \
    select("Origin"). \
    distinct(). \
    show()

In [ ]:
airlines. \
    join(airport_codes, airlines.Origin == airport_codes.IATA, "left"). \
    filter("IATA IS NULL"). \
    select("Origin"). \
    distinct(). \
    count()

## Solutions - Problem 5

Get the total number of flights from the airports that do not contain entries in airport-codes.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"

In [ ]:
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big')")

In [ ]:
airlines. \
    join(airport_codes, airlines.Origin == airport_codes.IATA, "left"). \
    filter("IATA IS NULL"). \
    count()

## Solutions - Problem 6

Get the total number of flights per airport that do not contain entries in airport-codes.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airport_codes_path = "/public/airlines_all/airport-codes"
airport_codes = spark. \
    read. \
    csv(airport_codes_path,
        sep="\t",
        header=True,
        inferSchema=True
       ). \
    filter("!(State = 'Hawaii' AND IATA = 'Big')")

In [ ]:
airlines. \
    join(airport_codes, airlines.Origin == airport_codes.IATA, "left"). \
    filter("IATA IS NULL"). \
    groupBy("Origin"). \
    count(). \
    show()